In [12]:
import cobra as cb

In [13]:
model = cb.io.read_sbml_model('/home/yashjonjale/Documents/iGEM/iGEM-IITB/FBA/Metabolic_Reconstructions/MODEL1108160000_url.xml')


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_12dgr120_c "1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_12dgr120_c "1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_12dgr140_c "1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <S

In [14]:
model1 = cb.io.read_sbml_model('/home/yashjonjale/Documents/iGEM/iGEM-IITB/FBA/Metabolic_Reconstructions/iECD_1391.xml')

In [15]:
model2 = cb.io.read_sbml_model('/home/yashjonjale/Documents/iGEM/iGEM-IITB/FBA/Metabolic_Reconstructions/ecoli_core.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl-phosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_2pg_c "D-Glycerate-2-phosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_3pg_c "3-Phospho-D-glycerate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_6pgc_c "6-Phospho-D-gluconate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_6pgl_c "6-phospho-D-glucono-1-5-lactone">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_ac_c "Acetate">
Use of FORMULA in the note

In [16]:
print(f"The reactions, metabolites and genes in the ecoli k12 bigger model are: {len(model.reactions)}, {len(model.metabolites)}, {len(model.genes)}")

The reactions, metabolites and genes in the ecoli k12 bigger model are: 2583, 1805, 1367


In [17]:
soln1 = model.optimize()
print(soln1)
soln2 = model1.optimize()
print(soln2)
soln3 = model2.optimize()
print(soln3)

<Solution 0.982 at 0x73c76106bf10>
<Solution 0.976 at 0x73c763f884f0>
<Solution infeasible at 0x73c763bbd390>


/home/yashjonjale/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [18]:
print(soln1.fluxes)
print(f"the final objective function value is {soln1.objective_value}")
print(type(soln1.fluxes))

DM_4CRSOL    0.000219
DM_5DRIB     0.000221
DM_AACALD    0.000000
DM_AMOB      0.000002
DM_MTHTHF    0.000440
               ...   
ZN2abcpp     0.000000
ZN2t3pp      0.000000
ZN2tpp       0.000335
ZNabcpp      0.000000
Zn2tex       0.000335
Name: fluxes, Length: 2583, dtype: float64
the final objective function value is 0.9823718127270189
<class 'pandas.core.series.Series'>


In the context of cobrapy and flux balance analysis (FBA), **shadow prices** refer to the dual values associated with the constraints in a linear programming problem. Here's how they are described on the web page you're viewing:

- **Shadow Prices** : These are part of the `Solution` object returned by the `Model.optimize()` function in cobrapy. They are represented as a pandas series indexed by the metabolite identifier¹[1].
- **Purpose**: Shadow prices provide insight into how much the objective value (such as biomass production) would change if the availability of a particular metabolite were to be increased or decreased slightly.
- **Interpretation**: A positive shadow price indicates that if the supply of the corresponding metabolite is increased by one unit, the objective value will increase by the shadow price value. Conversely, a negative shadow price suggests that increasing the supply of the metabolite would decrease the objective value.

This concept is crucial in metabolic engineering and systems biology for understanding the sensitivity of metabolic networks to changes in metabolite levels and identifying potential bottlenecks in metabolic pathways.

In [19]:
print(model)

iJO1366


In [20]:
print(model.objective.direction)
print(model.objective.direction)

max
max


### Example 1

Simulating biomass growth under aerobic conditions

- glucose uptake bound = 18.5 mmol/gDWh
- aerobic conditions, so oxygen intake -1000


In [23]:
#lets first verify a few things about the model

obj_dict = model.objective
print(obj_dict)

Maximize
1.0*Ec_biomass_iJO1366_core_53p95M - 1.0*Ec_biomass_iJO1366_core_53p95M_reverse_e94eb


In [25]:
#lets fix the bounds
glc_ex = model.reactions.get_by_id('EX_glc_LPAREN_e_RPAREN_')

o2_ex = model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_')

In [37]:
glc_ex.lower_bound =-18.5
glc_ex.upper_bound = 1000
o2_ex.lower_bound = -1000
o2_ex.upper_bound = 0


In [38]:
ex1_p1_soln = model.optimize()

In [39]:
print(ex1_p1_soln)
print(f"the final objective function value is {ex1_p1_soln.objective_value}")
# pri

<Solution 1.828 at 0x73c761071300>
the final objective function value is 1.8284931967096778


In [40]:
print(model.reactions.get_by_id('EX_glc_LPAREN_e_RPAREN_').flux)
print(model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_').flux)

-18.5
-32.04147285966901


### part of example 1

anaerobic conditions

take o2 exchange to be 0


In [41]:
o2_ex.lower_bound = 0
o2_ex.upper_bound = 0

In [42]:
ex1_p1_soln = model.optimize()

In [43]:
print(ex1_p1_soln)
print(f"the final objective function value is {ex1_p1_soln.objective_value}")
# pri

<Solution 0.476 at 0x73c76102f010>
the final objective function value is 0.47637149248915706


In [44]:
print(model.reactions.get_by_id('EX_glc_LPAREN_e_RPAREN_').flux)
print(model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_').flux)

-18.5
0.0
